In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import csv

with open("./ee.csv", 'r') as file:
    train_df = pd.read_csv(file)


In [3]:
train_df["CONTEXT"] = train_df["TITLE"] + ". " + train_df["ABSTRACT"]

In [4]:
try:
    train_df.drop(labels=['ID', 'TITLE', 'ABSTRACT'], axis = 1, inplace=True)
except:
    print("already been deleted")

In [5]:
train_df = train_df[["CONTEXT", 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance']]

In [6]:
target_list = ['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance']

In [7]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARING_RATE = 1e-05

In [8]:
from transformers import BertTokenizer, BertModel

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [10]:
example_text = "do like and subscribe my channel"
encoding = tokenizer.encode_plus(
    example_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt"
)

In [11]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = self.df["CONTEXT"]
        self.targets = self.df[target_list].values
    
    def __len__(self):
        return len(self.title)
    
    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
    
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        return{
            "input_ids": inputs["input_ids"].flatten(),
            "return_attention_mask": inputs["return_attention_mask"].flatten(),
            "return_token_type_ids": inputs["return_token_type_ids"].flatten(),
            "targets": torch.FloatTensor(self.targets[index])
        }
        

In [12]:
train_size = .8
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df.drop(train_df.index).reset_index(drop=True)

In [14]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
val_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    
)